# 2/3 We load files from S3 to build an AWS RDS Database

## First we load files from S3

In [5]:
import boto3
import pandas as pd

BUCKET_NAME = 'ylequere-jedha'
print(f"Loading Kayak csv files from S3 {BUCKET_NAME} bucket.")
print("\n!!! The credentials are needed to achieve this task. Please contact ylequere@gmail.com to obtain it !!!")
s3 = boto3.resource("s3")
bucket = s3.Bucket(BUCKET_NAME)

print("\n# Downloading locations_weather.csv from https://ylequere-jedha.s3.eu-west-3.amazonaws.com/02-Kayak/locations_weather.csv")
df_locations_weather = pd.read_csv(bucket.Object('02-Kayak/locations_weather.csv').get()['Body'])

print("\n# Downloading hotels.csv from https://ylequere-jedha.s3.eu-west-3.amazonaws.com/02-Kayak/hotels.csv")
df_hotels = pd.read_csv(bucket.Object('02-Kayak/hotels.csv').get()['Body'])

Loading Kayak csv files from S3 ylequere-jedha bucket.

!!! The credentials are needed to achieve this task. Please contact ylequere@gmail.com to obtain it !!!

# Downloading locations_weather.csv from https://ylequere-jedha.s3.eu-west-3.amazonaws.com/02-Kayak/locations_weather.csv

# Downloading hotels.csv from https://ylequere-jedha.s3.eu-west-3.amazonaws.com/02-Kayak/hotels.csv


In [6]:
df_locations_weather.head(5)

,Id,location,pop,perceived_temperature,display_name,lat,lon
0,0,Aigues Mortes,25,16,"Aigues-Mortes, Nîmes, Gard, Occitanie, France ...",43.565823,4.191284
1,1,Aix en Provence,15,15,"Aix-en-Provence, Bouches-du-Rhône, Provence-Al...",43.529842,5.447474
2,2,Amiens,33,12,"Amiens, Somme, Hauts-de-France, France métropo...",49.894171,2.295695
3,3,Annecy,24,10,"Annecy, Haute-Savoie, Auvergne-Rhône-Alpes, Fr...",45.899235,6.128885
4,4,Ariege,23,7,"Ariège, Occitanie, France métropolitaine, France",42.945537,1.406554


In [7]:
df_hotels.head(5)

,Id,title,score,desc,url,lat,lon,location
0,0,le coin des hirondelles,9.7,"Situé à Pontorson, l'établissement le coin des...",https://www.booking.com/hotel/fr/le-coin-des-h...,48.602864,-1.474069,Mont Saint Michel
1,1,L'ancien Presbytère d'Ardevon,9.6,"Situé à Pontorson, L'ancien Presbytère d'Ardev...",https://www.booking.com/hotel/fr/l-39-ancien-p...,48.603324,-1.476397,Mont Saint Michel
2,2,"Maison chaleureuse, spacieuse et familiale",9.5,"Dotée d'une connexion Wi-Fi gratuite, la Maiso...",https://www.booking.com/hotel/fr/maison-chaleu...,48.596577,-1.505204,Mont Saint Michel
3,3,Gîtes le Mont Desclos Saint Michel,9.4,"Situé à Beauvoir, à 5,7 km de l'abbaye du Mont...",https://www.booking.com/hotel/fr/gites-le-mont...,48.596656,-1.504449,Mont Saint Michel
4,4,Résidence Beauvoir le Mont-Saint-Michel (9 gît...,9.3,"Installée à 4,4 km de l'abbaye du Mont-Saint-M...",https://www.booking.com/hotel/fr/residence-bea...,48.597878,-1.508419,Mont Saint Michel


##  Files are loaded successfully !

## Second, we build the AWS MySQL DB

In [8]:
import pandas as pd
import os

# Saving data under MySQL DB in AWS RDS
from sqlalchemy import create_engine
DBUSER = 'admin'
DBHOST = 'kayak.cw9vzlqtne1z.eu-west-3.rds.amazonaws.com'
PORT = 3306
print(f"\n# Saving data under MySQL DB in AWS RDS as {DBUSER}:DBPASS@{DBHOST}:{PORT}")
print("\n !!! The KAYAK DB password is needed to achieve this. Please contact ylequere@gmail.com to obtain it !!!")
DBPASS = os.getenv('KAYAK_DB_PASSWORD')
engine = create_engine(f"mysql+pymysql://{DBUSER}:{DBPASS}@{DBHOST}:{PORT}/", echo=False)


# Saving data under MySQL DB in AWS RDS as admin:DBPASS@kayak.cw9vzlqtne1z.eu-west-3.rds.amazonaws.com:3306

 !!! The KAYAK DB password is needed to achieve this. Please contact ylequere@gmail.com to obtain it !!!


In [9]:
print("\n# Creating kayakdb")
engine.execute("CREATE DATABASE IF NOT EXISTS kayakdb")
engine.execute("USE kayakdb")
engine.execute("DROP TABLE IF EXISTS LOCATIONS_WEATHER")
engine.execute("DROP TABLE IF EXISTS HOTELS")


# Creating kayakdb


## DB is created and tables are dropped.

In [10]:
try:
    df_locations_weather.to_sql('LOCATIONS_WEATHER', engine, index=False)
# Unexpected error which not prevents to save data in DB
except AttributeError:
    pass

In [11]:
try:
    df_hotels.to_sql('HOTELS', engine, index=False)
# Unexpected error which not prevents to save data in DB
except AttributeError:
    pass

## Tables are created and fed in the DB.

## Let's test some query !

In [12]:
join_query = """SELECT L.location, L.pop, L.perceived_temperature, L.display_name, L.lat, L.lon
,  H.title, H.score, H.desc, H.url
FROM LOCATIONS_WEATHER L 
INNER JOIN HOTELS H 
ON L.location=H.location
"""
# Get result of join between location_weather and hotels on location into a list
print("\n# Get result of join between location_weather and hotels on location into a list (first row)")
l_w = engine.execute(join_query).fetchall()
print(l_w[0])
print(f"=> Size of query result : {len(l_w)} rows")

# Get result of join between location_weather and hotels on location into a DataFrame
print("\n# Get result of join between location_weather and hotels on location into a DataFrame (5 first rows)")
df_result = pd.read_sql(join_query, engine)
print(f"=> Size of query result: {len(df_result.index)} rows")


# Get result of join between location_weather and hotels on location into a list (first row)
('Mont Saint Michel', 54, 12, 'Mont Saint-Michel, Plateforme du Saut-\xadGaultier, Le Mont-Saint-Michel, Avranches, Manche, Normandie, France métropolitaine, 50170, France', 48.6359541, -1.511459954959514, 'le coin des hirondelles', 9.7, "Situé à Pontorson, l'établissement le coin des hirondelles propose un service de prêt de vélos, un jardin, une terrasse et une connexion Wi-Fi gratuite. ", 'https://www.booking.com/hotel/fr/le-coin-des-hirondelles.fr.html')
=> Size of query result : 1005 rows

# Get result of join between location_weather and hotels on location into a DataFrame (5 first rows)
=> Size of query result: 1005 rows


In [14]:
df_result.sample(20)

,location,pop,perceived_temperature,display_name,lat,lon,title,score,desc,url
379,Besancon,26,10,"Besançon, Doubs, Bourgogne-Franche-Comté, Fran...",47.238022,6.024362,Appartement hyper centre ville au calme,9.0,"Situé à Besançon, à 1 km de la gare de Besanço...",https://www.booking.com/hotel/fr/appartement-h...
234,Lille,39,11,"Lille, Nord, Hauts-de-France, France métropoli...",50.636565,3.063528,aux abord de Lille - Bord de deule,9.1,"Situé à Marquette-lès-Lille, à 7,7 km du centr...",https://www.booking.com/hotel/fr/aux-abord-de-...
574,Cassis,16,16,"Cassis, Marseille, Bouches-du-Rhône, Provence-...",43.214036,5.539632,SunSet Cassis,9.4,"Offrant une vue sur la mer, le SunSet Cassis p...",https://www.booking.com/hotel/fr/sunset-cassis...
590,Marseille,17,16,"Marseille, Bouches-du-Rhône, Provence-Alpes-Cô...",43.296174,5.369953,"Balcon Filant, Vue 180, Store, Netflix, Clim ,...",9.6,"Situé au cœur de Marseille, le Balcon Filant, ...",https://www.booking.com/hotel/fr/le-suffren-ba...
257,Strasbourg,9,11,"Strasbourg, Bas-Rhin, Grand Est, France métrop...",48.584614,7.750713,LE LOFT 67,9.5,"Situé dans le centre de Strasbourg, à moins de...",https://www.booking.com/hotel/fr/le-loft-67-st...
173,Paris,19,12,"Paris, Île-de-France, France métropolitaine, F...",48.858890,2.320041,Studio confort Vanves Paris Porte de Versailles,9.7,Le Studio confort Vanves Paris Porte de Versai...,https://www.booking.com/hotel/fr/studio-confor...
499,Lyon,24,12,"Lyon, Métropole de Lyon, Circonscription dépar...",45.757814,4.832011,Lyon Cosy Stay,9.5,"Situé à Lyon, à 600 mètres du musée des beaux-...",https://www.booking.com/hotel/fr/lyon-cosy-sta...
643,Avignon,21,15,"Avignon, Vaucluse, Provence-Alpes-Côte d'Azur,...",43.949249,4.805901,Le Clos Saluces,9.9,Le Clos Saluces possède un jardin fleuri aména...,https://www.booking.com/hotel/fr/le-clos-saluc...
357,Eguisheim,7,11,"Eguisheim, Colmar-Ribeauvillé, Haut-Rhin, Gran...",48.044797,7.307962,Gîte chez Patou,9.4,"Situé à Wintzenheim, à seulement 200 mètres de...",https://www.booking.com/hotel/fr/gite-chez-pat...
937,Biarritz,38,15,"Biarritz, Bayonne, Pyrénées-Atlantiques, Nouve...",43.471144,-1.552727,Résidence ADAGIO,9.3,"Dotée d'une connexion Wi-Fi gratuite, la Résid...",https://www.booking.com/hotel/fr/residence-ada...


## The KAYAK DB is in place !